# Clean Demographics Data

In [3]:
# tidy_province_wide_to_long.py
# Convert multi-header wide sheets (year/sex/age in columns) to tidy rows.
# Output columns: year, province, variable (sheet name), sex, age, value

import pandas as pd
import numpy as np
import re

INPUT_PATH = "/workarea/HDI_Italy/data/raw/Indicatori_demografici.xls" 
OUT_CSV    = "/workarea/HDI_Italy/data/processed/italy_provincial_tidy.csv"
OUT_XLSX   = "/workarea/HDI_Italy/data/processed/italy_provincial_tidy.xlsx"

def read_all_sheets(path):
    # Try read with up to 3 header rows to capture cases like: Year / Sex / Age
    # (Works even if fewer header rows are present; pandas fills NaNs.)
    return pd.read_excel(path, sheet_name=None, header=[0,1,2])

def most_stringy_column(df):
    # Choose the column that looks most like "territory" (mostly non-numeric text)
    best_col = df.columns[0]
    best_score = -1
    for col in df.columns:
        s = df[col]
        # if this column is multiindex (tuple) it's fine; the data is still a Series
        ser = s.dropna().astype(str).str.strip()
        if ser.empty:
            score = 0
        else:
            num_share = pd.to_numeric(ser.str.replace(",", ".", regex=False), errors="coerce").notna().mean()
            score = 1 - num_share  # higher => more string-like
        if score > best_score:
            best_score = score
            best_col = col
    return best_col

def detect_levels(data_cols):
    """
    Decide which column level is year / sex / age.
    Heuristics:
      - year level: shares many 4-digit tokens (19xx/20xx)
      - sex level: contains 'maschi', 'femmine', 'totale'
      - remaining -> age (if present)
    """
    nlevels = data_cols.nlevels
    year_level = sex_level = age_level = None

    # 1) Year detection
    best_year_score = -1
    for i in range(nlevels):
        vals = data_cols.get_level_values(i).astype(str)
        score = np.mean(vals.str.contains(r"(19|20)\d{2}", regex=True, na=False))
        if score > best_year_score:
            best_year_score = score
            year_level = i
    if best_year_score < 0.2:  # weak signal: treat "year" as None (will just carry raw label)
        year_level = None

    # 2) Sex detection among remaining levels
    sex_words = ("maschi", "femmine", "femmina", "maschio", "totale")
    best_sex_score = -1
    for i in range(nlevels):
        if i == year_level:
            continue
        vals = data_cols.get_level_values(i).astype(str).str.lower()
        score = np.mean(vals.str.contains("|".join(sex_words), regex=True, na=False))
        if score > best_sex_score:
            best_sex_score = score
            sex_level = i
    if best_sex_score < 0.2:
        sex_level = None

    # 3) Age = the other remaining level (if any)
    remaining = [i for i in range(nlevels) if i not in {year_level, sex_level}]

    if remaining:
        age_level = remaining[0]

    return year_level, sex_level, age_level

def extract_year(x):
    m = re.search(r"(19|20)\d{2}", str(x))
    return int(m.group(0)) if m else pd.NA

def clean_value(series):
    s = series.astype(str).str.replace("\u00A0", " ", regex=False).str.strip()
    s = s.str.replace(".", "", regex=False)      # remove thousands
    s = s.str.replace(",", ".", regex=False)     # decimal comma -> dot
    s = s.str.replace("%", "", regex=False)      # drop percent
    s = s.replace({"": np.nan, "-": np.nan, "n.d.": np.nan, "nd": np.nan})
    return pd.to_numeric(s, errors="coerce")

def run():
    # Read workbook
    # Note: for .xls you may need engine="xlrd"; for .xlsx engine="openpyxl".
    sheets = read_all_sheets(INPUT_PATH)

    all_out = []

    for sheet_name, df in sheets.items():
        if df is None or df.empty:
            continue

        # Drop fully empty rows/cols
        df = df.dropna(axis=1, how="all").dropna(axis=0, how="all")

        # Detect province/region column (territory)
        territory_col = most_stringy_column(df)

        # Set territory as index; remaining columns are data
        data = df.set_index(territory_col)

        # Ensure columns are a MultiIndex (even if single-level)
        if not isinstance(data.columns, pd.MultiIndex):
            data.columns = pd.MultiIndex.from_arrays([data.columns])

        # Identify which level is year / sex / age
        year_level, sex_level, age_level = detect_levels(data.columns)
        nlevels = data.columns.nlevels
        stack_levels = list(range(nlevels))  # stack all column levels

        # Stack to tidy
        tidy = data.stack(stack_levels).reset_index()

        # Rename columns
        rename_map = {tidy.columns[0]: "province"}  # index (territory)
        col_names = list(tidy.columns)

        # The stacked header levels start at position 1 and go on for nlevels
        level_cols = col_names[1:1+nlevels]
        for i, col in enumerate(level_cols):
            if i == year_level:
                rename_map[col] = "year"
            elif sex_level is not None and i == sex_level:
                rename_map[col] = "sex"
            elif age_level is not None and i == age_level:
                rename_map[col] = "age"
            else:
                # any extra header level -> keep a generic name
                rename_map[col] = f"attr_{i+1}"

        # The last column is the value
        rename_map[col_names[-1]] = "value"
        tidy = tidy.rename(columns=rename_map)

        # Clean / standardize fields
        if "year" in tidy.columns:
            tidy["year"] = tidy["year"].map(extract_year)
            tidy["year"] = pd.to_numeric(tidy["year"], errors="coerce").astype("Int64")

        if "sex" in tidy.columns:
            tidy["sex"] = tidy["sex"].astype(str).str.strip().str.title()

        if "age" in tidy.columns:
            tidy["age"] = tidy["age"].astype(str).str.strip()

        tidy["province"] = (
            tidy["province"].astype(str).str.strip()
            .str.replace(r"\s+", " ", regex=True).str.title()
        )
        tidy["value"] = clean_value(tidy["value"])
        tidy["variable"] = sheet_name  # <- sheet name as variable

        # Keep only the columns we care about
        keep_cols = ["year", "province", "variable", "sex", "age", "value"]
        existing = [c for c in keep_cols if c in tidy.columns]
        tidy = tidy[existing]

        # Remove empty rows (no year or no value)
        if "year" in tidy.columns:
            tidy = tidy.dropna(subset=["year"])
        tidy = tidy.dropna(subset=["value"])

        all_out.append(tidy)

    # Combine and save
    out = pd.concat(all_out, ignore_index=True) if all_out else pd.DataFrame(
        columns=["year","province","variable","sex","age","value"]
    )
    out = out.sort_values(["variable","province","year"], na_position="last").reset_index(drop=True)

    out.to_csv(OUT_CSV, index=False, encoding="utf-8")
    out.to_excel(OUT_XLSX, index=False)

    print("Done.")
    print("Rows:", len(out))
    print("Sheets processed (as 'variable'):", out["variable"].nunique() if not out.empty else 0)
    print(f"Wrote: {OUT_CSV} and {OUT_XLSX}")

if __name__ == "__main__":
    run()


/tmp/ipykernel_49923/1007139722.py:51: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  score = np.mean(vals.str.contains(r"(19|20)\d{2}", regex=True, na=False))
/tmp/ipykernel_49923/1007139722.py:51: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  score = np.mean(vals.str.contains(r"(19|20)\d{2}", regex=True, na=False))
/tmp/ipykernel_49923/1007139722.py:51: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  score = np.mean(vals.str.contains(r"(19|20)\d{2}", regex=True, na=False))
/tmp/ipykernel_49923/1007139722.py:122: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new i

Done.
Rows: 72670
Sheets processed (as 'variable'): 14
Wrote: /workarea/HDI_Italy/data/processed/italy_provincial_tidy.csv and /workarea/HDI_Italy/data/processed/italy_provincial_tidy.xlsx


# Clean Territories names

In [5]:
from standardize_italy_admin_names import standardize_territory

In [6]:
df = pd.read_csv('/workarea/HDI_Italy/data/processed/italy_provincial_tidy.csv')

In [ ]:
col = 'province'  # or 'territory'
std = df[col].apply(standardize_territory).apply(pd.Series)
df = pd.concat([df, std], axis=1)

In [11]:
df

,year,province,variable,age,value,sex,level,province_std,region_std,macro_std
0,2002,Abruzzo,crescita_naturale,Crescita naturale (per mille),-21.0,NaN,region,None,Abruzzo,Sud
1,2003,Abruzzo,crescita_naturale,Crescita naturale (per mille),-21.0,NaN,region,None,Abruzzo,Sud
2,2004,Abruzzo,crescita_naturale,Crescita naturale (per mille),-15.0,NaN,region,None,Abruzzo,Sud
3,2005,Abruzzo,crescita_naturale,Crescita naturale (per mille),-18.0,NaN,region,None,Abruzzo,Sud
4,2006,Abruzzo,crescita_naturale,Crescita naturale (per mille),-16.0,NaN,region,None,Abruzzo,Sud
...,...,...,...,...,...,...,...,...,...,...
72665,2020,Viterbo,tasso_di_fecondità_totale,Numero medio di figli per donna*,109.0,NaN,province,Viterbo,Lazio,Centro
72666,2021,Viterbo,tasso_di_fecondità_totale,Numero medio di figli per donna*,11.0,NaN,province,Viterbo,Lazio,Centro
72667,2022,Viterbo,tasso_di_fecondità_totale,Numero medio di figli per donna*,11.0,NaN,province,Viterbo,Lazio,Centro
72668,2023,Viterbo,tasso_di_fecondità_totale,Numero medio di figli per donna*,108.0,NaN,province,Viterbo,Lazio,Centro


## Completeness check

In [12]:
from completeness import (
    compute_coverage,
    find_gaps,
    list_missing_territories,
    plot_coverage_heatmap_plotly,
)

In [13]:
# Keep one slice (optional): only Totale sex if present
filters = {"sex": ["Totale", None]}

In [14]:
# Province-level coverage
cov_prov = compute_coverage(df, level="province", filters=filters, expected="all")

# Region-level coverage
cov_reg = compute_coverage(df, level="region", filters=filters, expected="all")

# Which variable-years have gaps?
gaps = find_gaps(cov_prov)  # coverage < 1
print(gaps.head(20))

Empty DataFrame
Columns: [level, variable, year, present_count, expected_count, missing_count, coverage]
Index: []


In [20]:
# Plot heatmap (subset variables if you have many)
# vars_to_plot = cov_prov["variable"].value_counts().head(20).index
vars_to_plot = df['variable'].unique().tolist()  # or e.g. ["crescita_naturale","tasso_di_fecondità_totale"]

fig = plot_coverage_heatmap_plotly(
    cov_prov,
    level="province",
    variables=vars_to_plot,
    annotate=False,        # set True to print percentages inside cells
    height=700,
)
fig.show()

ValueError: No data to plot after filtering.